# Local Govornment Area School Data Analysis

In [ ]:
import pandas as pd
from pathlib import Path

school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


